In [1]:
import numpy as np
import pandas as pd
import nltk
import re
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
from nltk.stem.snowball import SnowballStemmer

In [3]:
from sklearn.externals import joblib

In [4]:
from sklearn.cluster import KMeans

In [5]:
stemmer = SnowballStemmer('english')

In [6]:
def tokenize_and_stem(text):
    tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    for token in tokens:
        if re.search('[a-zA-Z0-9]',token):
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems

def tokenize_only(text):
    tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    for token in tokens:
        if re.search('[a-zA-Z0-9]',token):
            filtered_tokens.append(token)     
    return filtered_tokens

In [7]:
synopses_wiki = []
synopses_wiki = open('wiki_summaries2.txt',encoding='utf-8').read().split('\nBREAK\n')

In [8]:
book_names = open('book_names.txt').read().split('\n')

In [13]:
book_names

['To Kill a Mockingbird',
 '1984',
 'The Lord of the Rings (The Lord of the Rings, #1-3)',
 'The Catcher in the Rye',
 'The Great Gatsby',
 'The Lion, the Witch, and the Wardrobe (Chronicles of Narnia, #1)',
 'Lord of the Flies',
 'Animal Farm',
 'Catch-22 (Catch-22, #1)',
 'The Grapes of Wrath',
 'Gone with the Wind',
 'Slaughterhouse-Five',
 'Lolita',
 "One Flew Over the Cuckoo's Nest",
 'A Clockwork Orange',
 "Are You There God? It's Me, Margaret",
 'Watchmen',
 'Never Let Me Go',
 'Atonement',
 'Things Fall Apart (The African Trilogy, #1)',
 'Beloved',
 'Invisible Man',
 'Mrs. Dalloway',
 'The Sun Also Rises',
 'On the Road',
 'Possession',
 'The Big Sleep (Philip Marlowe, #1)',
 'A Passage to India',
 'I, Claudius (Claudius, #1)',
 'Their Eyes Were Watching God',
 'The Sound and the Fury',
 "All the King's Men",
 'The Blind Assassin',
 'Native Son',
 'To the Lighthouse',
 'Ragtime',
 "The French Lieutenant's Woman",
 'The Heart is a Lonely Hunter',
 'Light in August',
 'Naked Lunc

In [4]:
synopses_wiki

[' List of To Kill a Mockingbird characters  The story takes place during three years (1933–35) of the Great Depression in the fictional "tired old town" of Maycomb, Alabama, the seat of Maycomb County. It focuses on six-year-old Jean Louise Finch (nicknamed Scout), who lives with her older brother, Jeremy (nicknamed Jem), and their widowed father, Atticus, a middle-aged lawyer. Jem and Scout befriend a boy named Dill, who visits Maycomb to stay with his aunt each summer. The three children are terrified yet fascinated by their neighbor, the reclusive Arthur "Boo" Radley. The adults of Maycomb are hesitant to talk about Boo, and few of them have seen him for many years. The children feed one another\'s imagination with rumors about his appearance and reasons for remaining hidden, and they fantasize about how to get him out of his house. After two summers of friendship with Dill, Scout and Jem find that someone leaves them small gifts in a tree outside the Radley place. Several times th

In [20]:
synopses_wiki[89]

' edit  The novel follows the movements of William Bradshaw, its narrator, who meets a nervous-looking man named Arthur Norris on a train going from the Netherlands to Germany. As they approach the frontier William strikes up a conversation with Mr Norris, who wears an ill-fitting wig and carries a suspect passport.  William and Mr Norris succeed in crossing the frontier. Afterward, Mr Norris invites William to dinner and the two become friends. In Berlin they see each other frequently (including eating ham and eggs at the first class restaurant of Berlin Friedrichstraße railway station). Several oddities of Mr Norris\'s personal life are revealed, one of which is that he is a masochist. Another is that he is a Communist, which is dangerous in Hitler-era Germany. Other aspects of Mr Norris\'s personal life remain mysterious. He seems to run a business with an assistant Schmidt, who tyrannises him. Norris gets into more and more straitened circumstances and has to leave Berlin.  Berlin 

In [11]:
for i in range(len(synopses_wiki)):
    print(len(synopses_wiki[i]))

' edit  The story begins with Kathy, who describes herself as a "carer", talking about looking after organ donors. She has been a carer for almost twelve years at the time of narration, and she often reminisces about her time spent at Hailsham, a boarding school in England, where the teachers are known as guardians. Along with classes, they often emphasize the importance of being healthy to their students—smoking is considered to be taboo, almost on the level of a crime, and working in the vegetable garden is compulsory. The curriculum appears to be like that of any other school, but there is great encouragement for the students to produce art. The art is then displayed in an exhibition, and the best artwork is chosen by a woman known to the students as Madame. The students speculate that she keeps their work in a gallery.  boarding school  The story revolves around three Hailsham students: Kathy, and two others, Ruth and Tommy, who develop a close but complicated friendship. Kathy dev

In [9]:
stop_words = nltk.corpus.stopwords.words('english')

In [10]:
stemmer = SnowballStemmer('english')

remove text between square brackets and multiple spaces

In [11]:
for i in range(len(synopses_wiki)):
    synopses_wiki[i] = synopses_wiki[i].replace("'","")
    synopses_wiki[i] = re.sub("[[].*?[\]]","",synopses_wiki[i])
    synopses_wiki[i] = re.sub(" +"," ",synopses_wiki[i])

In [12]:
synopses_wiki[89]

' edit The novel follows the movements of William Bradshaw, its narrator, who meets a nervous-looking man named Arthur Norris on a train going from the Netherlands to Germany. As they approach the frontier William strikes up a conversation with Mr Norris, who wears an ill-fitting wig and carries a suspect passport. William and Mr Norris succeed in crossing the frontier. Afterward, Mr Norris invites William to dinner and the two become friends. In Berlin they see each other frequently (including eating ham and eggs at the first class restaurant of Berlin Friedrichstraße railway station). Several oddities of Mr Norriss personal life are revealed, one of which is that he is a masochist. Another is that he is a Communist, which is dangerous in Hitler-era Germany. Other aspects of Mr Norriss personal life remain mysterious. He seems to run a business with an assistant Schmidt, who tyrannises him. Norris gets into more and more straitened circumstances and has to leave Berlin. Berlin Berlin 

In [13]:
totalvocab_stemmed = []
totalvocab_tokenized = []
for i in synopses_wiki:
    allwords_stemmed = tokenize_and_stem(i)
    totalvocab_stemmed.extend(allwords_stemmed)
    
    allwords_tokenized = tokenize_only(i)
    totalvocab_tokenized.extend(allwords_tokenized)

In [14]:
vocab_data = pd.DataFrame({'words':totalvocab_tokenized }, index = totalvocab_stemmed)

In [15]:
vocab_data

,words
list,list
of,of
to,to
kill,kill
a,a
mockingbird,mockingbird
charact,characters
the,the
stori,story
take,takes


In [183]:
tfidf_vec = TfidfVectorizer(max_df=0.55,min_df=0.1,
                            stop_words='english',
                            tokenizer=tokenize_and_stem,
                            use_idf=True,
                            ngram_range=(1,3))

In [184]:
%time tfidf_mat = tfidf_vec.fit_transform(synopses_wiki)

Wall time: 32.2 s


In [182]:
tfidf_mat.shape

(100, 575)

In [156]:
kmeans = joblib.load('cluster1.pkl')

In [157]:
stemmer.stem('hotel\'s')

'hotel'

In [185]:
tfidf_mat.shape

(100, 570)

In [159]:
kmeans

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=4, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [172]:
tfidf_vec?

In [194]:
document_terms = tfidf_vec.get_feature_names

In [53]:
len(document_terms)

590

In [186]:
num = 4
kmeans = KMeans(n_clusters=num,n_init=20)

%time kmeans.fit(tfidf_mat)


Wall time: 4.11 s


KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=4, n_init=20, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [187]:
clusters = kmeans.labels_.tolist()

In [188]:
clusters

[1,
 0,
 2,
 3,
 3,
 1,
 1,
 2,
 2,
 2,
 2,
 3,
 2,
 2,
 2,
 1,
 3,
 1,
 2,
 1,
 1,
 1,
 0,
 2,
 3,
 0,
 1,
 1,
 0,
 1,
 2,
 0,
 0,
 1,
 2,
 3,
 1,
 2,
 1,
 2,
 1,
 1,
 1,
 2,
 1,
 1,
 2,
 0,
 2,
 1,
 2,
 1,
 0,
 0,
 1,
 2,
 1,
 2,
 2,
 1,
 2,
 2,
 2,
 1,
 2,
 2,
 3,
 2,
 2,
 0,
 2,
 3,
 3,
 1,
 2,
 2,
 3,
 2,
 2,
 1,
 0,
 1,
 1,
 3,
 1,
 1,
 2,
 2,
 2,
 2,
 3,
 3,
 2,
 2,
 2,
 2,
 0,
 3,
 0,
 2]

In [189]:
books_data = {'book_names':book_names, 'synopsis':synopses_wiki, 'cluster':clusters}

In [190]:
book_dataframe = pd.DataFrame(books_data,index=[clusters], columns=['book_names','synopsis','cluster'])

In [191]:
book_dataframe

,book_names,synopsis,cluster
1,To Kill a Mockingbird,List of To Kill a Mockingbird characters The ...,1
0,1984,edit Winston Smith is a man who lives in Airs...,0
2,"The Lord of the Rings (The Lord of the Rings, ...","edit Many years ago, two Hobbits, Smeagol and...",2
3,The Catcher in the Rye,"edit Holden Caulfield, a teenager from New Yo...",3
3,The Great Gatsby,edit The main events of the novel take place ...,3
1,"The Lion, the Witch, and the Wardrobe (Chronic...",edit too long excessively detailed help impro...,1
1,Lord of the Flies,"In the midst of a wartime evacuation, a Briti...",1
2,Animal Farm,"Old Major, the old boar on the Manor Farm, su...",2
2,"Catch-22 (Catch-22, #1)",edit The plotline follows the airmen of the 2...,2
2,The Grapes of Wrath,edit The narrative begins just after Tom Joad...,2


In [192]:
book_dataframe['cluster'].value_counts()

2    42
1    31
3    14
0    13
Name: cluster, dtype: int64

In [195]:
centroids = kmeans.cluster_centers_.argsort()[:,::-1]
for i in range(num):
    print("cluster %d words" %i)
    print()
    for ind in centroids[i,:7]:
#         print(ind)
        print(document_terms[ind])
#         print(' %s'%vocab_data[document_terms[ind]],end=",")

cluster 0 words



TypeError: 'method' object is not subscriptable